In [5]:
from modules.browser_control import BrowserControl
from facebook_register import FacebookRegister

bc = BrowserControl()
fr = FacebookRegister()
print(fr.browser_id)

2024/07/25 10:49:39 INFO     获取手机号码[16727961988]-->16727961988,怡萱,田,2000,3,3,xTK8QURkbMV3CVT

                    INFO     create browser:a9ff81bb8ea041578de749bd46914c77

2024/07/25 10:49:45 INFO     connect driver:{'success': True, 'data': {'ws':                                       
                             'ws://127.0.0.1:51162/devtools/browser/292bbc80-dc20-40b9-9a45-0b1b7d6145f9', 'http': 
                             '127.0.0.1:51162', 'coreVersion': '126', 'driver':                                    
                             'C:\\Users\\lqj\\AppData\\Roaming\\BitBrowser\\chromedriver\\126\\chromedriver.exe',  
                             'seq': 92, 'name': 'facebook-16727961988', 'remark':                                  
                             '16727961988,怡萱,田,2000,3,3,xTK8QURkbMV3CVT', 'pid': 34780}}

a9ff81bb8ea041578de749bd46914c77


In [2]:
fr.driver.get('https://mbasic.facebook.com')

In [3]:

from selenium.webdriver.common.by import By
helper = fr.helper
sign_up_btn = helper.find_or_fail(By.NAME,'sign_up')
sign_up_btn.click()


In [4]:
username_input = helper.find_or_fail(By.NAME,'reg_email__')
lastname_input = helper.find_or_fail(By.NAME,'lastname')
firstname_input = helper.find_or_fail(By.NAME,'firstname')
password_input = helper.find_or_fail(By.NAME,'reg_passwd__')
year_choose = helper.find_or_fail(By.NAME,'birthday_year')
month_choose = helper.find_or_fail(By.NAME,'birthday_month')
day_choose = helper.find_or_fail(By.NAME,'birthday_day')
sex_click = helper.find_or_fail(By.NAME,'sex')

username_input.send_keys(f'8617148869184')
lastname_input.send_keys(f'{fr.lastname}')
firstname_input.send_keys(f'{fr.firstname}')
password_input.send_keys(f'{fr.fb_password}')
year_choose.send_keys(f'{fr.year}')
month_choose.send_keys(f'{fr.month}')
day_choose.send_keys(f'{fr.day}')
sex_click.click()

In [ ]:
fr.sms.get_message()